In [1]:
import pandas as pd
import numpy as np
#import datetime
import dateutil
#import datetime as datetime
from datetime import datetime
from datetime import timedelta


In [14]:
today = datetime.now()
today2 = datetime.utcnow()
print(today)
print(today2)

t_interval = timedelta(days=0,hours=0,minutes=2)

t_now = datetime.utcnow()
t_prec = t_now - t_interval

print("timestamp now: {}, timestampOld: {}".format(t_now.strftime('%Y-%m-%d %H:%M:%S'),t_prec.strftime('%Y-%m-%d %H:%M:%S')))

2021-09-18 16:37:21.501358
2021-09-18 14:37:21.501422
timestamp now: 2021-09-18 14:37:21, timestampOld: 2021-09-18 14:35:21


In [6]:
df = pd.DataFrame(np.array([["20:20:20","20:20:21","20:20:22"],["ciao","arrivederci","oh vieni!"],["positive","negative","neutral"]]),columns=["Tweet Data","Tweet Text","tag"])

print(df)

  Tweet Data   Tweet Text        tag
0   20:20:20     20:20:21   20:20:22
1       ciao  arrivederci  oh vieni!
2   positive     negative    neutral


In [2]:
d = {
    "Tweet Data":["20:20:20","20:20:21","20:20:22","20:20:23","20:20:24","20:20:25"],
    "Tweet Text":["ciao","arrivederci","oh vieni!","ciao","arrivederci","oh vieni!"],
    "tag":["positive","positive","neutral","positive","negative","neutral"]
}

columns=["Tweet Data","Tweet Text","tag"]
df = pd.DataFrame(d)
df['Tweet Data'] = pd.to_datetime(df['Tweet Data'], format='%H:%M:%S')
print(df)

df.set_index(pd.DatetimeIndex(df["Tweet Data"]))
#del df["Tweet Data"]
print(df)


           Tweet Data   Tweet Text       tag
0 1900-01-01 20:20:20         ciao  positive
1 1900-01-01 20:20:21  arrivederci  positive
2 1900-01-01 20:20:22    oh vieni!   neutral
3 1900-01-01 20:20:23         ciao  positive
4 1900-01-01 20:20:24  arrivederci  negative
5 1900-01-01 20:20:25    oh vieni!   neutral
           Tweet Data   Tweet Text       tag
0 1900-01-01 20:20:20         ciao  positive
1 1900-01-01 20:20:21  arrivederci  positive
2 1900-01-01 20:20:22    oh vieni!   neutral
3 1900-01-01 20:20:23         ciao  positive
4 1900-01-01 20:20:24  arrivederci  negative
5 1900-01-01 20:20:25    oh vieni!   neutral


In [4]:
values = {
    "positive": 0,
    "negative": 0,
    "neutral": 0,
}


list_tags = df['tag']



for value in list_tags:
    if(value == 'neutral'):
        values["neutral"] += 1

    if(value == 'negative'):
        values["negative"] += 1

    if(value == 'positive'):
        values["positive"] += 1

print(values)

{'positive': 3, 'negative': 1, 'neutral': 2}


In [39]:
#a quanto pare non devo passare 
#una stringa di data ma un oggetto data

values = {'dates':  ['20190902','20190913','20190921'],
          'status': ['Opened','Opened','Closed']
          }

df = pd.DataFrame(values, columns = ['dates','status'])

df['dates'] = pd.to_datetime(df['dates'], format='%Y%m%d')

print (df)
print (df.dtypes)

       dates  status
0 2019-09-02  Opened
1 2019-09-13  Opened
2 2019-09-21  Closed
dates     datetime64[ns]
status            object
dtype: object


In [23]:

#time_now = pd.to_datetime("2020-08-25 10:13:10")
#time_past = pd.to_datetime("2020-08-25 10:11:10")

now_time = datetime.strptime("2020-08-25 10:18:49",'%Y-%m-%d %H:%M:%S')
interval_time = timedelta(days=0,hours=0,minutes=2)
past_time = now_time - interval_time

print(past_time.strftime('%Y-%m-%d %H:%M:%S'))

df_db = pd.read_csv("CleanedTweets.csv",index_col=False)
df_db['Tweet Datetime'] = pd.to_datetime(df_db['Tweet Datetime'], format='%Y-%m-%d %H:%M:%S')

del(df_db['index']) #Se non levavo questa colonna non funzionava il groupBy di pandas. 
                    #I conti non venivano fatti a modo...
    
#df_db.set_index("Tweet Datetime")
print(df_db)

time_mask = (df_db['Tweet Datetime'] > past_time) & \
            (df_db['Tweet Datetime'] <= now_time)


df2 = df_db[time_mask]


print(df2)

df3 = df2.reset_index()
print(df3)
#df2.set_index(pd.DatetimeIndex(df2["Tweet Datetime"]))
#del(df2[index])

2020-08-25 10:16:49
         Tweet Datetime               Tweet Text       tag
0   2020-08-25 10:10:11  Apple phone is good bad   neutral
1   2020-08-25 10:10:12  Apple phone is good bad   neutral
2   2020-08-25 10:10:13  Apple phone is good bad   neutral
3   2020-08-25 10:10:14  Apple phone is good bad  negative
4   2020-08-25 10:10:15  Apple phone is good bad   neutral
..                  ...                      ...       ...
395 2020-08-25 10:16:46  Apple phone is good bad  positive
396 2020-08-25 10:16:47  Apple phone is good bad  negative
397 2020-08-25 10:16:48  Apple phone is good bad   neutral
398 2020-08-25 10:16:49  Apple phone is good bad  positive
399 2020-08-25 10:16:50  Apple phone is good bad   neutral

[400 rows x 3 columns]
         Tweet Datetime               Tweet Text      tag
399 2020-08-25 10:16:50  Apple phone is good bad  neutral
   index      Tweet Datetime               Tweet Text      tag
0    399 2020-08-25 10:16:50  Apple phone is good bad  neutral


In [24]:

result = df2.groupby([pd.Grouper(key='Tweet Datetime',freq='20s'),'tag']).count()

print(result)

result2 = result.unstack(fill_value=0)
print(result2)

result3 = result2.stack()
print(result3)

result4 = result3.reset_index()
print(result4)

result5 = result4.rename(columns={"Tweet Text":"Num of '{}' mentions".format("Apple"),
                                 "Tweet Datetime":"Time in UTC"})
print(result5)

time_series = result5["Time in UTC"][result5['tag']=='positive'].reset_index(drop=True)

print(time_series)



                             Tweet Text
Tweet Datetime      tag                
2020-08-25 10:16:40 neutral           1
                    Tweet Text
tag                    neutral
Tweet Datetime                
2020-08-25 10:16:40          1
                             Tweet Text
Tweet Datetime      tag                
2020-08-25 10:16:40 neutral           1
       Tweet Datetime      tag  Tweet Text
0 2020-08-25 10:16:40  neutral           1
          Time in UTC      tag  Num of 'Apple' mentions
0 2020-08-25 10:16:40  neutral                        1
Series([], Name: Time in UTC, dtype: datetime64[ns])
